<a href="https://colab.research.google.com/github/taniokah/Sports_Analysis/blob/master/Player_Tracking_for_ICCP2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import cv2 #　opencvのインポート
import matplotlib.pyplot as plt # matplotlib(描画用)

print(cv2.__version__)

3.4.3


In [2]:
!pip install pytube
!pip install ffmpeg

  Stored in directory: /root/.cache/pip/wheels/b6/68/c3/a05a35f647ba871e5572b9bbfc0b95fd1c6637a2219f959e7a
Successfully built ffmpeg


In [0]:
from pytube import YouTube

yt = YouTube("https://youtu.be/l00CwQ6HI_o")


In [0]:
yt.streams.all()
#yt.streams.filter(progressive=False).desc().all()
#yt.streams.get_by_itag(313)
#yt.streams.first().download()
yt.streams.get_by_itag(313).download()

In [0]:
!rm -rf screen_caps
!mkdir screen_caps

In [0]:
#動画ファイルを読み込む
#file_name = u"20190217 徳島大学医学部 vs 吉野クラブ 2nd No2.mp4"
#file_name = u"20190224 鹿屋体育大学 vs 鹿児島高専 1st.mp4"
file_name = u"/content/20190224 鹿屋体育大学 vs 鹿児島高専 1st.webm"
video = cv2.VideoCapture(file_name)

#フレームレート数の取得
frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
print('frame_count = ' + str(frame_count))

#フレームレート(1フレームの時間単位はミリ秒)の取得
frame_rate = int(video.get(cv2.CAP_PROP_FPS))
print('frame_rate = ' + str(frame_rate))

# 画像を保存する
dir_name = "screen_caps"
for i in range(frame_count):
  _, frame = video.read()
  if i % 330 == 0: # 10秒で1枚
    file_name = dir_name+ "/" + str(i) + ".png"
    cv2.imwrite(file_name, frame)
  if i > 4000:
    break
